#### 1. 필요한 라이브러리 및 패키지 임포트

In [1]:
import pandas as pd
import numpy as np

# selenium의 webdriver를 사용하기 위한 import
from selenium import webdriver

# selenium으로 키를 조작하기 위한 import
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# 페이지 로딩을 기다리는데에 사용할 time 모듈 import
import time
from bs4 import BeautifulSoup as bs

#### 2. 크롤링

In [10]:
# 크롬드라이버 실행
driver = webdriver.Chrome()

# 크롬 드라이버에 네이버 카페 메인 url 주소 넣고 실행
driver.get('INPUT_NaverCafe')

search_box = driver.find_element(By.ID, "topLayerQueryInput") # 카페 검색박스

search_box.send_keys('INPUT_검색 키워드') # 검색어 입력
search_box.send_keys(Keys.RETURN)
time.sleep(random.randint(1,3)) # 요청시간 랜덤 조정으로 크롤링 차단 방지

- 검색 옵션 설정

In [12]:
driver.switch_to.frame(driver.find_element(By.XPATH, "//*[@id='cafe_main']")) # select component frame으로 이동

driver.find_element(By.XPATH, '//*[@id="listSizeSelectDiv"]/a').click() # 한 페이지에 뜨는 게시물 수 선택 버튼 클릭
driver.find_element(By.XPATH, '//*[@id="listSizeSelectDiv"]/ul/li[7]/a').click() # 50개씩 노출하기 버튼 클릭

- 게시글 제목, 본문, 작성시간, 댓글 추출

In [ ]:
num_page = 100 # 총 페이지 수
page = 1 # 시작 페이지 설정

df = pd.DataFrame(columns=['time', 'board_type', 'title', 'content','comment'])

while page <= num_page:
    print(f'page {page}')
    for i in range(1, 51): # 한 번에 보이는 게시글 수
        print(f'i : {i}, 게시글 번호 : {len(df) + 1}')
        idx = (page * 50) + i
        try:
            driver.switch_to.frame(driver.find_element(By.XPATH, "//*[@id='cafe_main']"))
        except:
            pass

        try:
            xpath = f'//*[@id="main-area"]/div[5]/table/tbody/tr[{i}]/td[1]/div[2]/div/a' # 게시글로 이동
            driver.find_element(By.XPATH, xpath).click()
            time.sleep(1)  # 페이지 로딩 시간
            driver.implicitly_wait(1)
        except:
            driver.back() # 이전 페이지로 돌아가기
            time.sleep(1)
            break

        try:
            board_type = driver.find_elements(By.XPATH, '//*[@id="app"]/div/div/div[2]/div[1]/div[1]/div/a') # 게시판 가져오기
            board_type = board_type[0].text

            if ('벼룩' in board_type) or ('이벤트' in board_type): # 게시글의 게시판이 벼룩 또는 이벤트일때 제외
                driver.back() # 이전 페이지로 돌아가기
                time.sleep(1)
                continue
        except:
            driver.back() # 이전 페이지로 돌아가기
            time.sleep(1)
            continue

        date = driver.find_elements(By.XPATH, '//*[@id="app"]/div/div/div[2]/div[1]/div[2]/div[2]/div[2]/span[1]') # 날짜 가져오기
        date = date[0].text
        print(date)
        
        title = driver.find_elements(By.XPATH, '//*[@id="app"]/div/div/div[2]/div[1]/div[1]/div/div/h3') # 제목 가져오기
        title = title[0].text

        try:
            content = driver.find_elements(By.XPATH, '//*[starts-with(@id, "SE")]') # 내용 가져오기
            content = content[0].text
            comment = driver.find_elements(By.XPATH, '//*[@id="app"]/div/div/div[2]/div[2]/div[5]/div[2]/ul') # 댓글 가져오기
            comment = comment[0].text

        except Exception as e: # 예외처리
            try:
                content = driver.find_elements(By.XPATH, '//*[starts-with(@id, "app")]')
                content = content[0].text
            except Exception as e: # 예외처리
                content = '내용없음'
                print(f"내용없음 오류 post {i} on page {page}: {e}")

        new_row = [date, board_type, title, content, comment]
        df.loc[len(df)] = new_row

        driver.back() # 이전 페이지로 돌아가기
        time.sleep(1)

    driver.switch_to.frame(driver.find_element(By.XPATH, "//*[@id='cafe_main']"))
    print(page % 10) 

    if (page % 10) == 0: # 현재 페이지가 10,20.. 단위일 때
        if (page // 10) > 1: # 현재 페이지가 20 이상
            next_page_button = driver.find_element(By.XPATH, "//*[@id='main-area']/div[7]/a[12]")
        else: # 현재 페이지가 10
            next_page_button = driver.find_element(By.XPATH, "//*[@id='main-area']/div[7]/a[11]")
    else: 
        if (page // 10) > 0: # 현재 페이지가 1~9
            next_page_button = driver.find_element(By.XPATH, f'//*[@id="main-area"]/div[7]/a[{(page % 10)+2}]')
        else: # 현재 페이지가 11이상
            next_page_button = driver.find_element(By.XPATH, f'//*[@id="main-area"]/div[7]/a[{(page % 10)+1}]')
    
    next_page_button.click() # 다음 페이지 이동 버튼 클릭
    page += 1
    time.sleep(1)

#### 3. 데이터 저장

In [20]:
df.to_csv('NaverBlog.csv', index=False, encoding='utf-8-sig')